In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import logging
from sklearn.ensemble import RandomForestClassifier

In [2]:
dataset = pd.read_csv('q1.csv')
dataset= dataset[dataset['ans'].notna()]#ignore NAN values
# print(dataset.head())
# print(len(answer))

In [3]:
#classify training and test set
from sklearn.model_selection import train_test_split
train,test = train_test_split(dataset,test_size=0.15,random_state=42)

In [4]:
print(train.head())

   img_id                                                ans  marks
76  IMG77  ସମାନତା ସନ୍ଧି ପ୍ରଥା ଜେନେରାଲ ଡେଲହାଉସି ପ୍ରଣୟନ କରି...    4.5
39  IMG40  ଭାରତରେ ଇଂଗ୍ରେଜ଼ ମାନଙ୍କ ପ୍ରଭାବ ବିସ୍ତାର କରିବା ପା...    4.0
0   IMG01  ଇଂଗରେଜକଂ ସାମରାଜ୍ୟ କୁ ସୁଦୃଢ କରିବା ପାଇଁ ୱେଲସେଲି ...    5.0
79  IMG80  ସାମନ୍ତ ସନ୍ଧି ୱେଲସେଲି ପ୍ରଣୟନ କରିଥଲେ ଇଂଗ୍ରେଜ଼ ମା...    4.5
46  IMG47  ସାମନ୍ତ ସନ୍ଧିକୁ ଇଂଗ୍ରେଜ଼ ଲର୍ଡ଼ ଡେଲହାଉସି ପ୍ରଚଳନ ...    0.5


In [5]:
print(test)

   img_id                                                ans  marks
48  IMG49  ଭାରତରେ ସାମ୍ରାଜ୍ୟ ବିସ୍ତାର କରିବ ତଥା ବାଣିଜ୍ୟ ପ୍ରସ...    2.0
21  IMG22  ଇଂଗ୍ରେଜ଼ ମାନଂକ ବିନା ଅନୁମତିରେ କୌଣସି ଦେଶ ସହ ବାଣି...    3.0
5   IMG06  ଭାରତରେ ଇଂଗ୍ରେଜ଼ ମାନଂକ ସାମ୍ରାଜ୍ୟ ବିସ୍ତାର ପାଇଁ ୱ...    5.0
10  IMG11  ୱେଲସେଲି ସାମନ୍ତ ସନ୍ଧି ପ୍ରଚଳନ କରିଥିଲେ|ତାଙ୍କର ଅନ୍...    2.0
58  IMG59  ଭାରତରେ ଇଂଗ୍ରେଜ଼ ମାନଙ୍କ ସାମ୍ରାଜ୍ୟ ବୃଦ୍ଧି ପାଇଁ ୱ...    5.0
52  IMG53  ଭାରତରେ ଇଂଗ୍ରେଜ଼ ମାନଙ୍କ ପ୍ରଭୁତ୍ୱ ବିସ୍ତାର କାରିବା...    4.5
78  IMG79  ଇଂଗ୍ରେଜ଼ ମାନେ ସାମ୍ରାଜ୍ୟ ବୃଦ୍ଧି କରିବା ପାଇଁ ୱେଲସ...    4.0
6   IMG07  ୱେଲସେଲି ଇଂଗ୍ରେଜ଼ ଶାସନର ପ୍ରସାର ପାଇଁଏକ ପ୍ରଥା ପ୍ର...    5.0
81  IMG82  ଇଂଗ୍ରେଜ଼ ମାନେ ଭାରତରେ ବାଣିଜ୍ୟ କରିବାକୁ ଆସିଥିଲେ|ଭ...    2.0
16  IMG17  ଭାରତରେ ରାଜ୍ୟ ବିସ୍ତାର କରିବା ପାଇଁ ଲର୍ଡ଼ ୱେଲସେଲି ...    5.0
34  IMG35  ସାମନ୍ତ ସନ୍ଧି ପ୍ରଥା ପ୍ରଚଳନ କରିଥିଲେ କାରଣ ସେ ଭାରତ...    0.0


In [6]:
from inltk.inltk import tokenize,get_embedding_vectors #importing tokenizer using inltk
def sentence_tokenization(answer):
    token = []
    for ans in answer:
        token.append(tokenize(str(ans),'or'))
    return token

In [7]:
answer = []
for ans in train['ans']:
    answer.append(ans)
token=[]
print("Tokenization of training set")
token = sentence_tokenization(answer)
# print(token)

Tokenization of training set


In [8]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
        level=logging.INFO)

In [9]:
    num_features = 400    # Word vector dimensionality
    min_word_count = 5   # Minimum word count
    num_workers = 4       # Number of threads to run in parallel
    window = 5         # Context window size
    downsampling = 1e-3   # Downsample setting for frequent words

In [10]:
#     print("Training Word2Vec model...")
#     model = Word2Vec(token, workers=num_workers, \
#                 size=num_features, min_count = min_word_count, \
#                 window = window, sample = downsampling, seed=1)

#     # If you don't plan to train the model Any further, calling
#     # init_sims will make the model much more memory-efficient.
#     model.init_sims(replace=True)
#     print("MODEL",model)
#     # It can be helpful to create a meaningful model name and
#     # save the model for later use. You can load it later using Word2Vec.load()
#     model_name = "word2vec_model"
#     model.save(model_name)

2020-05-03 01:58:12,580 : INFO : collecting all words and their counts
2020-05-03 01:58:12,581 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-03 01:58:12,584 : INFO : collected 662 word types from a corpus of 6956 raw words and 56 sentences
2020-05-03 01:58:12,585 : INFO : Loading a fresh vocabulary
2020-05-03 01:58:12,587 : INFO : effective_min_count=5 retains 199 unique words (30% of original 662, drops 463)
2020-05-03 01:58:12,588 : INFO : effective_min_count=5 leaves 6186 word corpus (88% of original 6956, drops 770)
2020-05-03 01:58:12,590 : INFO : deleting the raw counts dictionary of 662 items
2020-05-03 01:58:12,591 : INFO : sample=0.001 downsamples 93 most-common words
2020-05-03 01:58:12,592 : INFO : downsampling leaves estimated 3157 word corpus (51.0% of prior 6186)
2020-05-03 01:58:12,594 : INFO : estimated required memory for 199 words and 400 dimensions: 736300 bytes
2020-05-03 01:58:12,595 : INFO : resetting layer weights
2020-05-03 0

Training Word2Vec model...
MODEL Word2Vec(vocab=199, size=400, alpha=0.025)


In [11]:
def featureVec(words, model, num_features):
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the ans and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    #
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
#     print(featureVec)
    return featureVec

In [12]:
def avgFeatureVecs(reviews, model, num_features):
    # Initialize a counter
    counter = 0.
    #
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    #
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 10th review
       if counter%10. == 0.:
           print("Review %d of %d" % (counter, len(reviews)))
       #
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[int(counter)] = featureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    print(reviewFeatureVecs)
    return reviewFeatureVecs

In [13]:
# trainVec = avgFeatureVecs( token, model, num_features )

Review 0 of 56
Review 10 of 56
Review 20 of 56
Review 30 of 56
Review 40 of 56
Review 50 of 56
[[ 0.01893   0.031015  0.013522 -0.03856  ...  0.012261 -0.043516  0.034974  0.046051]
 [ 0.018951  0.03109   0.013488 -0.038462 ...  0.012214 -0.04347   0.034959  0.046107]
 [ 0.018916  0.031144  0.013583 -0.038507 ...  0.01215  -0.043496  0.034871  0.046188]
 [ 0.018877  0.030978  0.013518 -0.038557 ...  0.012275 -0.043537  0.034934  0.046087]
 ...
 [ 0.018962  0.031086  0.013452 -0.038442 ...  0.012161 -0.043488  0.035098  0.045973]
 [ 0.019039  0.030906  0.013539 -0.038622 ...  0.012129 -0.043519  0.035029  0.045999]
 [ 0.01896   0.031014  0.013548 -0.038417 ...  0.012293 -0.043424  0.034947  0.046112]
 [ 0.019005  0.031014  0.013469 -0.038522 ...  0.012222 -0.043547  0.03495   0.046089]]


/home/iamsid2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [14]:
# print(trainVec)

[[ 0.01893   0.031015  0.013522 -0.03856  ...  0.012261 -0.043516  0.034974  0.046051]
 [ 0.018951  0.03109   0.013488 -0.038462 ...  0.012214 -0.04347   0.034959  0.046107]
 [ 0.018916  0.031144  0.013583 -0.038507 ...  0.01215  -0.043496  0.034871  0.046188]
 [ 0.018877  0.030978  0.013518 -0.038557 ...  0.012275 -0.043537  0.034934  0.046087]
 ...
 [ 0.018962  0.031086  0.013452 -0.038442 ...  0.012161 -0.043488  0.035098  0.045973]
 [ 0.019039  0.030906  0.013539 -0.038622 ...  0.012129 -0.043519  0.035029  0.045999]
 [ 0.01896   0.031014  0.013548 -0.038417 ...  0.012293 -0.043424  0.034947  0.046112]
 [ 0.019005  0.031014  0.013469 -0.038522 ...  0.012222 -0.043547  0.03495   0.046089]]


In [15]:
# len(trainVec)

56

In [16]:
def inltkWordVec(asnwer):
    wordVec = []
    for ans in answer:
        wordVec.append(get_embedding_vectors(str(t),'or'))
    return wordVec

In [17]:
# itrainVec = inltkWordVec(answer)
# print(itrainVec)

In [18]:
# for i in itrainVec:
#     print(len(i))

In [19]:
answer_test = []
for ans in test['ans']:
    answer_test.append(str(ans))
token_test=[]
print("Tokenization of test set")
token_test = sentence_tokenization(answer_test)
# print(token_test)

Tokenization of test set


In [20]:
testVec = avgFeatureVecs(token_test, model, num_features)

Review 0 of 11
Review 10 of 11
[[ 0.019078  0.03101   0.013404 -0.038566 ...  0.012094 -0.04347   0.035063  0.046147]
 [ 0.019006  0.031069  0.013403 -0.038459 ...  0.012283 -0.043509  0.034956  0.046052]
 [ 0.018999  0.031042  0.013469 -0.038472 ...  0.01228  -0.043454  0.034968  0.046105]
 [ 0.018898  0.031005  0.013498 -0.038683 ...  0.012263 -0.043543  0.034936  0.046063]
 ...
 [ 0.018981  0.031034  0.013548 -0.038469 ...  0.012213 -0.043485  0.034988  0.046098]
 [ 0.01908   0.031007  0.013487 -0.03851  ...  0.012207 -0.043508  0.035006  0.046047]
 [ 0.018999  0.030961  0.013443 -0.038511 ...  0.012177 -0.04351   0.034986  0.046065]
 [ 0.019202  0.031122  0.013693 -0.038508 ...  0.012093 -0.043376  0.034953  0.046018]]


/home/iamsid2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [21]:
testVec[0].shape

(400,)

In [22]:
# itestVec = inltkWordVec(answer_test)
# print(itestVec)

In [23]:
# print("Length Vec Test",len(itestVec))
# print("Gensim Length")
# for t in testVec:
#       print(len(t))
# print("INLTK Length")
# for l in itestVec:
#     print(len(l))

In [24]:
def pad(l, content, width):
    l.extend([content] * (width - len(l)))
    return l

# pad(itrainVec,0,300)

In [25]:
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit(trainVec, train["marks"].astype(int))

# Test & extract results
# result = forest.predict(np.array(xtestVec,dtype="float32"))

Fitting a random forest to labeled training data...


In [26]:
# Test & extract results
result = forest.predict(testVec)
# print(result)

In [27]:
#Calculate accuracy

test_mark = []
for i in test['marks'].astype(int):
    test_mark.append(i)
print("Actual Mark")
test_mark = np.array(test_mark)
print(test_mark)
print("Predicted Mark")
print(result)
from sklearn.metrics import accuracy_score
accuracy_score(test_mark, result)

Actual Mark
[2 3 5 2 5 4 4 5 2 5 0]
Predicted Mark
[4 5 5 4 5 5 4 5 5 5 5]


0.45454545454545453

In [1]:
#Creating a CNN model
#step1-importing tf
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D